In [2]:
## Envinroment preparation

First line if **magic** enabling matplotlib inline plots 

In [3]:
%matplotlib inline

Then we have round of inports:
* **pandas** is our main data storage module
* **glob** and **os** are used for filename manipulations

In [4]:
import pandas as pd
import glob
import os

## Files index and identification

In [5]:
filenames = [ os.path.splitext(wholeFilename)[0] for wholeFilename in 
             [ os.path.basename(wholePath) for wholePath in glob.glob("../input/2*.xlsx") ] ]

In [6]:
dataFiles = pd.DataFrame({"filename": filenames})
dataFiles["year"], dataFiles["pollutant"], dataFiles["resolution"] = dataFiles["filename"].str.split('_', 2).str

In [7]:
dataFiles.head()

,filename,year,pollutant,resolution
0,2010_Pb(PM10)_24g,2010,Pb(PM10),24g
1,2007_As(PM10)_24g,2007,As(PM10),24g
2,2005_PM2.5_24g,2005,PM2.5,24g
3,2009_PM2.5_1g,2009,PM2.5,1g
4,2011_NH4+(PM2.5)_24g,2011,NH4+(PM2.5),24g


In [8]:
dataFiles["year"].value_counts()

2012    35
2011    35
2014    35
2013    34
2015    29
2010    25
2009    24
2008    24
2007    18
2006    18
2005    17
2004    17
2003    17
2002    14
2001    11
2000     6
Name: year, dtype: int64

In [9]:
dataFiles["pollutant"].value_counts()

NO2            32
SO2            31
PM10           29
C6H6           28
PM2.5          22
O3             16
NOx            16
Cd(PM10)       15
Ni(PM10)       15
BaP(PM10)      15
Pb(PM10)       14
As(PM10)       14
CO             13
IP(PM10)        8
BjF(PM10)       8
BbF(PM10)       8
BkF(PM10)       8
BaA(PM10)       8
DBahA(PM10)     7
formaldehyd     5
Na+(PM2.5)      4
NH4+(PM2.5)     4
NO3-(PM2.5)     4
Ca2+(PM2.5)     4
OC(PM2.5)       4
Mg2+(PM2.5)     4
Cl              4
EC(PM2.5)       4
SO42            4
K+(PM2.5)       4
PM25            2
Hg(TGM)         2
DBah(PM10)      1
depozycja       1
Jony            1
Name: pollutant, dtype: int64

In [10]:
dataFiles["resolution"].value_counts()

24g            236
1g             113
(PM2.5)-24g      4
(PM2.5)_24g      4
w_PM25_24g       1
Name: resolution, dtype: int64

## Fixing data files identification

In [11]:
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)-24g", 'pollutant'] = "Cl_(PM2.5)"
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)-24g", 'resolution'] = "24g"

In [12]:
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)_24g", 'pollutant'] = "SO42_(PM2.5)"
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)_24g", 'resolution'] = "24g"

In [13]:
dataFiles.ix[dataFiles["resolution"] == "w_PM25_24g", 'pollutant'] = "Jony_w_PM25"
dataFiles.ix[dataFiles["resolution"] == "w_PM25_24g", 'resolution'] = "24g"

Now **resolution** column should be correct:

In [14]:
dataFiles["resolution"].value_counts()

24g    245
1g     113
Name: resolution, dtype: int64

Lets also fix worngly labelled pollutant:

In [15]:
dataFiles.ix[dataFiles["pollutant"] == "PM25", 'pollutant'] = "PM2.5"

In [16]:
dataFiles.describe()

,filename,year,pollutant,resolution
count,359,359,359,358
unique,359,16,34,2
top,2012_NO2_24g,2012,NO2,24g
freq,1,35,32,245


There is still one empty cell in **resolution** column. Lets identify it:

In [17]:
dataFiles[dataFiles["resolution"].isnull()]

,filename,year,pollutant,resolution
312,2015_depozycja,2015,depozycja,NaN


After manually examinign **2015_depozycja** file I found that it cointains new type of data, which will be useless in planned analysis. I decided to remove it from working memory. 

In [18]:
dataFiles.drop(dataFiles[dataFiles["filename"] == "2015_depozycja"].index, inplace=True)

In [19]:
dataFiles.describe()

,filename,year,pollutant,resolution
count,358,358,358,358
unique,358,16,33,2
top,2012_NO2_24g,2012,NO2,24g
freq,1,35,32,245


## Looking for worst measuring station for each pollutant in 2015

In [20]:
importantPollutants = ["PM10", "PM2.5", "O3", "NO2", "SO2", "C6H6", "CO"]
pollutants2015 = dataFiles[(dataFiles["year"] == "2015") & (dataFiles["resolution"] == "1g") & 
                           (dataFiles["pollutant"].isin(importantPollutants))]

In [21]:
pollutants2015

,filename,year,pollutant,resolution
14,2015_NO2_1g,2015,NO2,1g
106,2015_CO_1g,2015,CO,1g
138,2015_SO2_1g,2015,SO2,1g
141,2015_O3_1g,2015,O3,1g
207,2015_C6H6_1g,2015,C6H6,1g
310,2015_PM10_1g,2015,PM10,1g
326,2015_PM25_1g,2015,PM2.5,1g


In [22]:
from tqdm import tqdm

In [23]:
from collections import Counter

In [24]:
#worstStation = {}
#for index, dataRow in tqdm(pollutants2015.iterrows(), total=len(pollutants2015.index)):
#    dataFromFile = pd.read_excel("../input/" + dataRow["filename"] + ".xlsx", skiprows=[1,2])
#    dataFromFile = dataFromFile.rename(columns={"Kod stacji":"Godzina"})
#    dataFromFile = dataFromFile.set_index("Godzina")
#    worstStation[dataRow["pollutant"]] = dataFromFile.max().sort_values(ascending = False).index[0]

## Building one big data frame

In [25]:
pollutants = importantPollutants
years = sorted(list(dataFiles["year"].unique()))
#years = ["2006"]

In [26]:
pollutantsYears = dataFiles[(dataFiles["year"].isin(years)) & (dataFiles["resolution"] == "1g") & 
                           (dataFiles["pollutant"].isin(pollutants))]

In [27]:
pollutantsYears

,filename,year,pollutant,resolution
3,2009_PM2.5_1g,2009,PM2.5,1g
5,2005_PM10_1g,2005,PM10,1g
8,2008_PM10_1g,2008,PM10,1g
10,2008_NO2_1g,2008,NO2,1g
12,2007_O3_1g,2007,O3,1g
14,2015_NO2_1g,2015,NO2,1g
15,2004_NO2_1g,2004,NO2,1g
24,2009_NO2_1g,2009,NO2,1g
31,2005_C6H6_1g,2005,C6H6,1g
33,2009_CO_1g,2009,CO,1g


In [28]:
bigDataFrame = pd.DataFrame()
for dataYear in years: 
    print(dataYear)
    yearDataFrame = pd.DataFrame()
    for index, dataRow in tqdm(pollutantsYears[pollutantsYears["year"] == dataYear].iterrows(), total=len(pollutantsYears[pollutantsYears["year"] == dataYear].index)):
        #print(dataRow["filename"])
        data = pd.read_excel("../input/" + dataRow["filename"] + ".xlsx", skiprows=[1,2])
        data = data.rename(columns={"Kod stacji":"Hour"})

        year = int(dataRow["year"])
        rng = pd.date_range(start = str(year) + '-01-01 01:00:00', end = str(year+1) + '-01-01 00:00:00', freq='H')

        # workaround for 2006_PM2.5_1g
        try:
            data["Hour"] = rng
        except ValueError:
            print("File {} has some mess with timestamps".format(dataRow["filename"]))
            continue

        data = data.set_index("Hour")
        data = data.stack()
        data = pd.DataFrame(data, columns=[dataRow["pollutant"]])
        data.index.set_names(['Hour', 'Station'], inplace=True)

        yearDataFrame = pd.concat([yearDataFrame, data], axis=1)
    
    bigDataFrame = bigDataFrame.append(yearDataFrame)

  0%|          | 0/2 [00:00<?, ?it/s]

2000


  0%|          | 0/3 [00:00<?, ?it/s]

2001


  0%|          | 0/3 [00:00<?, ?it/s]

...

100%|██████████| 7/7 [03:01<00:00, 28.32s/it]


2014


  0%|          | 0/7 [00:00<?, ?it/s]

2015


100%|██████████| 7/7 [03:16<00:00, 26.36s/it]


In [ ]:
bigDataFrame.to_pickle("../output/bigDataFrame.pkl")

In [ ]:
data1 = pd.read_excel("../input/2015_PM10_1g.xlsx", skiprows=[1,2])
data2 = pd.read_excel("../input/2015_PM25_1g.xlsx", skiprows=[1,2])

In [ ]:
data1 = data1.rename(columns={"Kod stacji":"Hour"})
data2 = data2.rename(columns={"Kod stacji":"Hour"})

In [ ]:
rng = pd.date_range(start = '2015-01-01 01:00:00', end = '2016-01-01 00:00:00', freq='H')
data1["Hour"] = rng
data2["Hour"] = rng

In [ ]:
data1 = data1.set_index("Hour")
data2 = data2.set_index("Hour")

In [ ]:
data1 = data1.stack()
data2 = data2.stack()

In [ ]:
data1 = pd.DataFrame(data1, columns=["PM10"])
data2 = pd.DataFrame(data2, columns=["PM25"])

In [ ]:
data1.index.set_names(['Hour', 'Station'], inplace=True)
data2.index.set_names(['Hour', 'Station'], inplace=True)

In [ ]:
dataMerged = pd.concat([data1, data2], axis=1)

In [ ]:
dataMerged.tail()

## Working with stations 

In [ ]:
stations = pd.read_excel("../input/Metadane_wer20160914.xlsx")

In [ ]:
nazywStacji = set(dane.columns.values)

In [ ]:
stacje = stacje.set_index("Nr")

In [ ]:
stacje[(stacje["Stary Kod stacji"]).isin(nazywStacji) | (stacje["Kod stacji"]).isin(nazywStacji)]

In [ ]:
interesujaceStacje = stacje[(stacje["Stary Kod stacji"]).isin(nazywStacji) | (stacje["Kod stacji"]).isin(nazywStacji)]

In [ ]:
interesujaceStacje

In [ ]:
interesujaceStacje.shape

In [ ]:
interesujaceStacje[[u'WGS84 \u03bb E',u'WGS84 \u03c6 N']]

In [ ]:
wspolrzedne = interesujaceStacje[[u'WGS84 \u03bb E',u'WGS84 \u03c6 N']].values

In [ ]:
wspolrzedne[:,1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(wspolrzedne[:,0], wspolrzedne[:,1])

In [ ]:
import folium
map_osm = folium.Map(location=[52.069167, 19.480556], zoom_start=6)

In [ ]:
map_osm

In [ ]:
interesujaceStacje.index

In [ ]:
for index, row in interesujaceStacje.iterrows():
    print row['Nazwa stacji']
    folium.Marker([row[u'WGS84 \u03c6 N'], row[u'WGS84 \u03bb E']], popup=row['Nazwa stacji']).add_to(map_osm)

In [ ]:
map_osm

In [ ]:
jeden_dzien = dane[dane.index == "2000-06-12 08:00:00"]

In [ ]:
do_interpolacji = pd.melt(jeden_dzien)

In [ ]:
do_interpolacji.rename(columns={"variable":"Stary Kod stacji"}, inplace=True)

In [ ]:
final = do_interpolacji.merge(interesujaceStacje[[u'WGS84 \u03bb E',u'WGS84 \u03c6 N', "Stary Kod stacji"]])

In [ ]:
x = final[u'WGS84 \u03bb E'].values
y = final[u'WGS84 \u03c6 N'].values
z = final[u'value'].values

In [ ]:
x

In [ ]:
import numpy as np
from scipy.interpolate import griddata

In [ ]:
xi = np.linspace(x.min(),x.max(),100)
yi = np.linspace(y.min(),y.max(),200)

xi = np.append(xi,x)
xi.sort()
yi = np.append(yi,y)
yi.sort()

zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')

In [ ]:
(x,y), z

In [ ]:
zi

In [ ]:
# contour the gridded data, plotting dots at the randomly spaced data points.
CS = plt.contour(xi,yi,zi)
CS = plt.contourf(xi,yi,zi)
plt.colorbar() # draw colorbar
# plot data points.
plt.scatter(x,y,marker='o',c='b',s=5)
plt.show()

In [ ]:
from folium import plugins

In [ ]:
nxpoints = (x.max() - x.min()) / .001
nypoints = (y.max() - y.min()) / .001

In [ ]:
xi = np.linspace(x.min(),x.max(),int(nxpoints))
yi = np.linspace(y.min(),y.max(),int(nypoints))

In [ ]:
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')

In [ ]:
print(xi.shape)
print(yi.shape)
np.isnan(zi[4502,5000])

In [ ]:
from tqdm import tqdm

In [ ]:
xlist = []
ylist = []
zlist = []

In [ ]:
for xelement in tqdm(range(xi.shape[0])):
    for yelement in range(yi.shape[0]):
        if np.isnan(zi[yelement,xelement]):
            pass
        else:
            #tmpData = pd.DataFrame()
            #tmpData["x"] = xi[xelement]
            xlist.append(xi[xelement])
            ylist.append(yi[yelement])
            zlist.append(zi[yelement,xelement])
            #tmpData["y"] = yi[yelement]
            #tmpData["z"] = zi[yelement,xelement]
            #dataForHeatmap.append(tmpData, ignore_index=True)

In [ ]:
dataForHeatmap = pd.DataFrame({"x":xlist, "y":ylist, "z":zlist})

In [ ]:
dataForHeatmap

In [ ]:
#plugins.HeatMap(zip(ylist, xlist, zlist)).add_to(map_osm)

In [ ]:
#map_osm

In [ ]:
file

In [ ]:
[ basename(wholeFilename) for wholeFilename in glob.glob("../input/2*.xlsx") ]


In [ ]:
["asdfasdf", "asdfa", "asdf"]

In [ ]:
data = pd.read_excel("../input/2015_PM10_1g.xlsx", skiprows=[1,2])

In [ ]:
print("data1[\"Hour\"].head():")
print(data1["Hour"].head())
print("data1[\"Hour\"].tail():")
print(data1["Hour"].tail())

In [ ]:
year